In [1]:
import pandas as pd
import ast

In [20]:
RAW_DATA_PATH: str = 'data/preprocessed_datasets/dataset_maske_binary_unbalanced.csv'
EXPORTED_DATA_PATH: str = 'data/parquet_files'

TRAIN_SPLIT: float = 0.8
RANDOM_STATE: int = 42

In [21]:
raw: pd.DataFrame = (
    pd.read_csv(
        RAW_DATA_PATH,
        skiprows=[1],
        index_col=[0],
        dtype={
            'text': 'string',
            'label': 'category'
        },
        converters={
            'span': ast.literal_eval
        }
    )
    .rename_axis('idx')
)
raw

,text,label,span
idx,,,
61ae371b-65c5-4a47-83e1-f129090ccaec,"Ich lerne Unechtes und Masken zu erkennen, abz...",web2,"[23, 28]"
8b1f28b1-c896-43af-a134-a5c5e21ebfe5,Die beruhigende Maske von Avril wurde speziell...,web2,"[16, 21]"
c7e90d4d-e5e0-4c26-ae9f-d1e8a9c9f645,"Kräfte, die in den Schatten gehen mussten, die...",web2,"[130, 135]"
7357580e-18f7-4ba5-89d8-5e9a03c2b986,Genau diese Pflanzen sind auch in der Maske en...,web2,"[38, 43]"
44949207-cf5c-41be-9de2-d608610d1c18,Sogar die hochgezogene Augenbraue hab’ ich gez...,web2,"[78, 83]"
...,...,...,...
de92135b-bf02-4655-956d-5bfc7c838900,Sie hatte ihre bis dahin kritische Haltung geg...,corona,"[146, 151]"
5fbba903-2b5f-45a4-966c-e03b8fef8a86,"Die Organisation warnte aber erneut, dass solc...",corona,"[49, 54]"
5c567be3-7dd6-4567-8c53-e6975702dabf,"Etwa, wenn die Masken oft angefasst oder zeitw...",corona,"[15, 20]"


In [22]:
raw.dtypes

text     string[python]
label          category
span             object
dtype: object

In [23]:
raw.label.unique()

['web2', 'corona']
Categories (2, object): ['corona', 'web2']

In [24]:
raw.label.value_counts()

label
web2      9746
corona    4983
Name: count, dtype: int64

In [25]:
(
    raw
    .apply(lambda row: row.text[row.span[0]:row.span[1]], axis=1)
    .value_counts()
)

Maske    14718
            11
Name: count, dtype: int64

In [26]:
train_split = raw.sample(
    frac=TRAIN_SPLIT, 
    random_state=RANDOM_STATE
)
train_split.to_parquet(f'{EXPORTED_DATA_PATH}/maske_bu_train.parquet')

In [27]:
test_split = raw.loc[
    raw.index.difference(train_split.index)
]
test_split.to_parquet(f'{EXPORTED_DATA_PATH}/maske_bu_test.parquet')